In [1]:
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
import keras
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint



Using TensorFlow backend.


In [2]:
IMG_PATH = "data/"
print(os.listdir(IMG_PATH))

['.ipynb_checkpoints', 'validation_set', 'training_set']


In [3]:
train_path = IMG_PATH + "training_set/"
validation_path = IMG_PATH + "validation_set/"

In [4]:
def number_of_imgs(path):
    print(path)
    for value in os.listdir(path):
        print(value, "has", len(os.listdir(path + value)), "imgs")

In [5]:
number_of_imgs(train_path)
number_of_imgs(validation_path)

data/training_set/
hemmorhage_data has 80 imgs
non_hemmorhage_data has 80 imgs
data/validation_set/
hemmorhage_data has 21 imgs
non_hemmorhage_data has 20 imgs


In [6]:
img_width, img_height = 224, 224
nb_train_samples = 160
nb_validation_samples = 41
epochs = 25
batch_size = 4
checkpoint_filepath = "checkpoint/model2_val_loss-{val_loss:.2f}.h5"

In [7]:
from keras.applications.inception_v3 import InceptionV3
conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))

W0827 14:04:41.743421 140435732875008 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 14:04:41.755340 140435732875008 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 14:04:41.758116 140435732875008 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 14:04:41.774682 140435732875008 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please

87916544/87910968 [==============================] - 15s 0us/step


In [8]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [9]:
#for layer in conv_base.layers:
    #layer.trainable = False

In [10]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [11]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())                                    
model.add(Dense(64))                                    
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))                                     
model.add(Activation('sigmoid'))            

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

W0827 14:05:11.935281 140435732875008 deprecation.py:506] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0827 14:05:11.964849 140435732875008 deprecation_wrapper.py:119] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0827 14:05:11.971364 140435732875008 deprecation.py:323] From /home/h8953/tensorflow_gpu_env/lib/python3.5/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has t

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                3276864   
_________________________________________________________________
activation_95 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_96 (Activation)   (None, 1)                 0         
Total para

In [13]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    rescale=1. / 255,                   #Scaling RGB values from "0-255" to "0-1" 
    zoom_range=0.2,                 #Random zoom between 0 and 0.2
    horizontal_flip=True) #Random horizontal flip
               


test_datagen = ImageDataGenerator(rescale=1. / 255) #Scaling RGB values from "0-255" to "0-1". 


# Generates batches of image data for training

train_generator = train_datagen.flow_from_directory(
    train_path,                                       #Train data directory
    target_size=(img_width, img_height),                  #Image size
    batch_size=batch_size,
    class_mode="binary")                                  #Amount of classes, binary if 2 classes, else categorical


validation_generator = test_datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary")


# Lowering learning rate if no progress is made.

reduce_learning_rate = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=5,
    min_lr=0.00001,
    verbose=1)


# Defining checkpoint that saves the best model or model"s weights based on the value that we are monitoring.

checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    #save_weights_only=True,
    mode="min")


# Defining callbacks which includes checkpoint and reducing learning rate on plateau.

callbacks = [checkpoint, reduce_learning_rate]

# Training CNN

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,         
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks)

Found 160 images belonging to 2 classes.
Found 41 images belonging to 2 classes.
Epoch 1/25
40/40 [==============================] - 19s 483ms/step - loss: 1.3028 - acc: 0.5562 - val_loss: 1.0082 - val_acc: 0.5500

Epoch 00001: val_loss improved from inf to 1.00819, saving model to checkpoint/model2_val_loss-1.01.h5
Epoch 2/25
40/40 [==============================] - 5s 131ms/step - loss: 0.8009 - acc: 0.6062 - val_loss: 0.5627 - val_acc: 0.7568

Epoch 00002: val_loss improved from 1.00819 to 0.56272, saving model to checkpoint/model2_val_loss-0.56.h5
Epoch 3/25
40/40 [==============================] - 5s 128ms/step - loss: 0.6294 - acc: 0.7000 - val_loss: 0.5306 - val_acc: 0.5946

Epoch 00003: val_loss improved from 0.56272 to 0.53056, saving model to checkpoint/model2_val_loss-0.53.h5
Epoch 4/25
40/40 [==============================] - 5s 128ms/step - loss: 0.5452 - acc: 0.7188 - val_loss: 0.5525 - val_acc: 0.6486

Epoch 00004: val_loss did not improve from 0.53056
Epoch 5/25
40/40 [